# bokeh

* interaktywne wizualizacje
* nowatorskie grafiki
* renderowanie w przeglądarce webowej
* serwer
* strumieniowanie danych, duże dane
* (zazwyczaj) nie ma potrzeby znania technologii webowych (HTML, JavaScript, CSS)
* integracja z jupyter notebook, numpy, pandas, matplotlib, seaborn, ...
* trzy poziomy API (bokeh.models, bokeh.plotting, bokeh.charts)
* wbudowany system statycznych typów danych
* nie tylko Python (R - rbokeh, Julia - bokeh.jl, Scala - bokeh-scala, ...)
* możliwość rozszerzania funkcjonalność biblioteki
* licencja: BSD

## jak to działa?

Python (lub R, Julia, Scala) -> JSON + (HTML, JS, CSS) -> przeglądarka

## bokeh vs. matplotlib

* przeglądarka webowa vs. qt/gkt + png,pdf,svg,...
* mało stabilne API vs. wysoko stabilne API
* wiele implementacji vs. tylko Python

## planowana funkcjonalność

* generowanie statycznych grafik (png/jpeg/svg) (integracja z node.js)
* LaTeX math

## przykłady

* http://demo.bokehplots.com
* http://cecp.mit.edu

## kontakt

* strona: https://bokeh.pydata.org
* repozytorium: https://github.com/bokeh/bokeh
* lista mailingowa: bokeh@continuum.io

## statystyki

* github
 * 3600+ obserwatorów
 * 700+ forków
* lista mailingowa
 * 400+ członków
 * 150+ wiadomości (listopad 2015)
* liczba pobrań
 * conda: ~21k miesięcznie
 * pip: ~8k miesięcznie

In [ ]:
from bokeh.io import output_notebook
output_notebook()

In [ ]:
from bokeh.plotting import *

x = list(range(1, 20))
y = [ xi**2 for xi in x ]

p = figure()
p.line(x, y, line_color="red")
show(p)

In [ ]:
import numpy as np
from bokeh.plotting import *

N = 20
x = np.random.normal(size=N)
y = np.random.normal(size=N)

minval = min(x.min(), y.min())*1.1
maxval = max(x.max(), y.max())*1.1

mids = (x + y)/2

p = figure()
p.line([minval, maxval], [minval, maxval], color='gray', line_width=3)
p.segment(mids, mids, x, y, color='gray', alpha=0.6, line_width=2, line_dash='dashed')
p.scatter(x, y, marker='circle', color='red', size=6)
show(p)